In [1]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import os
import plotly.express as px
import json
from urllib.request import urlopen

In [2]:
# Setting the Data files
counties_population_path = os.path.join("Resources","co-est2019-annres.xlsx")
covid_counties_path = os.path.join("Resources","us-counties.csv")

In [3]:
# Read the population data and the case results
population_metadata = pd.read_excel(counties_population_path)
case_results = pd.read_csv(covid_counties_path)

In [4]:
population_metadata

,Geographic Area,2018,2019
0,United States,326687501,328239523
1,".Autauga County, Alabama",55533,55869
2,".Baldwin County, Alabama",217855,223234
3,".Barbour County, Alabama",24872,24686
4,".Bibb County, Alabama",22367,22394
...,...,...,...
3138,".Sweetwater County, Wyoming",42858,42343
3139,".Teton County, Wyoming",23269,23464
3140,".Uinta County, Wyoming",20292,20226
3141,".Washakie County, Wyoming",7877,7805


In [5]:
case_results

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
...,...,...,...,...,...,...
615474,2020-10-09,Sweetwater,Wyoming,56037.0,393,2
615475,2020-10-09,Teton,Wyoming,56039.0,656,1
615476,2020-10-09,Uinta,Wyoming,56041.0,378,2
615477,2020-10-09,Washakie,Wyoming,56043.0,122,6


##Population Data

In [7]:
# Extract the county and state information from "Geographic Area" column
population_metadata["county"] = ""
population_metadata["state"] = ""
for index, row in population_metadata.iterrows():
    county_state =  row["Geographic Area"].split(",")
    population_metadata.loc[index,"county"] = county_state[0].split(" County")[0].lstrip(".")
    try:
        population_metadata.loc[index,"state"] = county_state[1].lstrip(" ")
    except:
        population_metadata.loc[index,"state"] = ""
population_metadata

,Geographic Area,2018,2019,county,state
0,United States,326687501,328239523,United States,
1,".Autauga County, Alabama",55533,55869,Autauga,Alabama
2,".Baldwin County, Alabama",217855,223234,Baldwin,Alabama
3,".Barbour County, Alabama",24872,24686,Barbour,Alabama
4,".Bibb County, Alabama",22367,22394,Bibb,Alabama
...,...,...,...,...,...
3138,".Sweetwater County, Wyoming",42858,42343,Sweetwater,Wyoming
3139,".Teton County, Wyoming",23269,23464,Teton,Wyoming
3140,".Uinta County, Wyoming",20292,20226,Uinta,Wyoming
3141,".Washakie County, Wyoming",7877,7805,Washakie,Wyoming


In [8]:
# Remove "Geographic Area" column
del population_metadata["Geographic Area"]

In [9]:
population_metadata

,2018,2019,county,state
0,326687501,328239523,United States,
1,55533,55869,Autauga,Alabama
2,217855,223234,Baldwin,Alabama
3,24872,24686,Barbour,Alabama
4,22367,22394,Bibb,Alabama
...,...,...,...,...
3138,42858,42343,Sweetwater,Wyoming
3139,23269,23464,Teton,Wyoming
3140,20292,20226,Uinta,Wyoming
3141,7877,7805,Washakie,Wyoming


In [10]:
population_metadata.loc[population_metadata["state"] == ""]

,2018,2019,county,state
0,326687501,328239523,United States,
